In [1]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import BPR, SLRC
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [4]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [5]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [6]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [7]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=5,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:12<00:00, 598.50it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4508.39it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 3953.99it/s]


In [8]:
def objective(trial):
    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": trial.suggest_categorical("emb_size", [32, 64, 128]),
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-12 09:29:35,501] A new study created in memory with name: no-name-d95d3175-61c7-422f-9fd9-bea34b30e576


Epoch 1:


Batch loss = 0.615568: 100%|██████████| 8889/8889 [01:37<00:00, 91.11it/s] 



Epoch 2:


Batch loss = 0.885531: 100%|██████████| 8889/8889 [01:35<00:00, 93.01it/s]


...


Epoch 4:


Batch loss = 0.38924: 100%|██████████| 4445/4445 [00:57<00:00, 77.34it/s]



Epoch 5:


Batch loss = 0.363346: 100%|██████████| 4445/4445 [00:57<00:00, 77.60it/s]


100%|██████████| 7357/7357 [01:25<00:00, 85.93it/s]
[I 2023-04-12 11:49:49,619] Trial 24 finished with value: 0.10904024662631921 and parameters: {'emb_size': 64, 'batch_size': 64, 'lr': 0.0006250653189707479, 'l2_reg_coef': 0.0028470312329004627}. Best is trial 21 with value: 0.1093370103339033.


Test SLRC (calculate scores for different seeds):

In [11]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 7358/7358 [00:11<00:00, 641.34it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3669.10it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4187.58it/s]


In [12]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": study.best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___0
Epoch 1:


Batch loss = 0.647844: 100%|██████████| 4445/4445 [00:54<00:00, 81.41it/s]


Evaluation (dev):



100%|██████████| 7357/7357 [01:24<00:00, 86.86it/s]


 {'precision': 0.05306510806035069, 'recall': 0.11760657494409904, 'ndcg': 0.10338726165353773}
Epoch 2:


...


Batch loss = 0.3681: 100%|██████████| 4445/4445 [00:56<00:00, 79.26it/s]


Evaluation (dev):



100%|██████████| 7357/7357 [01:26<00:00, 85.00it/s]


 {'precision': 0.05593312491504689, 'recall': 0.12913710209741236, 'ndcg': 0.10902443637702317}



100%|██████████| 7357/7357 [01:25<00:00, 85.88it/s]


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.06205790403697159,
 'recall': 0.14575089650712814,
 'ndcg': 0.11939254118955867}

# TaoBao

Fix seed:

In [14]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [15]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [16]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [17]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=2,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:32<00:00, 305.85it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 28939.10it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 16245.89it/s]


In [18]:
def objective(trial):
    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": trial.suggest_categorical("emb_size", [32, 64, 128]),
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-21 04:44:33,608] A new study created in memory with name: no-name-db970ecf-aabd-4706-bcd7-62daebe89b2b


Epoch 1:


Batch loss = 0.65354: 100%|██████████| 1522/1522 [00:14<00:00, 106.38it/s]



Epoch 2:


Batch loss = 0.394244: 100%|██████████| 1522/1522 [00:16<00:00, 94.53it/s] 


...

Epoch 1:


Batch loss = 0.674537: 100%|██████████| 1522/1522 [00:15<00:00, 98.01it/s] 



Epoch 2:


Batch loss = 0.674023: 100%|██████████| 1522/1522 [00:15<00:00, 98.32it/s] 


100%|██████████| 9307/9307 [02:55<00:00, 52.97it/s]
[I 2023-04-21 06:05:54,297] Trial 24 finished with value: 0.07317208512053477 and parameters: {'emb_size': 32, 'batch_size': 32, 'lr': 3.0351973315936807e-05, 'l2_reg_coef': 0.014726211366418478}. Best is trial 10 with value: 0.07391601599813025.


Test SLRC (calculate scores for different seeds):

In [21]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 10092/10092 [00:37<00:00, 271.23it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 31350.15it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 27222.16it/s]


In [22]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": study.best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___0
Epoch 1:


Batch loss = 0.671844: 100%|██████████| 191/191 [00:03<00:00, 54.47it/s]


Evaluation (dev):



100%|██████████| 9307/9307 [02:54<00:00, 53.33it/s]


 {'precision': 0.0104222628129365, 'recall': 0.09888077074603345, 'ndcg': 0.07300048930102894}
Epoch 2:


...


Batch loss = 0.671451: 100%|██████████| 191/191 [00:03<00:00, 49.65it/s]


Evaluation (dev):



100%|██████████| 9307/9307 [02:49<00:00, 54.90it/s]


 {'precision': 0.010433007413774578, 'recall': 0.09893449375022384, 'ndcg': 0.07252946776558236}



100%|██████████| 9307/9307 [02:51<00:00, 54.34it/s]


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.011578381863113787,
 'recall': 0.11183338705633752,
 'ndcg': 0.08009273036056093}

# Dunnhumby

Fix seed:

In [24]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [25]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [26]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [27]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=1,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:08<00:00, 289.10it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:12<00:00, 190.26it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 197.05it/s]


In [28]:
def objective(trial):
    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": trial.suggest_categorical("emb_size", [32, 64, 128]),
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-04-22 23:34:33,097] A new study created in memory with name: no-name-d4585873-f78e-43eb-af1f-f37c35d96eb3


Epoch 1:


Batch loss = 0.447194: 100%|██████████| 60237/60237 [29:10<00:00, 34.41it/s]


100%|██████████| 2357/2357 [03:43<00:00, 10.56it/s]
[I 2023-04-23 00:07:26,805] Trial 0 finished with value: 0.15148375345370244 and parameters: {'emb_size': 32, 'batch_size': 32, 'lr': 0.011018509458263562, 'l2_reg_coef': 0.00032161219616553475}. Best is trial 0 with value: 0.15148375345370244.


Epoch 1:


...

Batch loss = 0.601167: 100%|██████████| 7530/7530 [04:11<00:00, 29.90it/s]


100%|██████████| 2357/2357 [03:39<00:00, 10.72it/s]
[I 2023-04-23 05:35:08,729] Trial 23 finished with value: 0.16857128960145432 and parameters: {'emb_size': 32, 'batch_size': 256, 'lr': 0.0002504075415812634, 'l2_reg_coef': 0.04256733701472282}. Best is trial 22 with value: 0.16899702698754962.


Epoch 1:


Batch loss = 0.575684: 100%|██████████| 7530/7530 [04:21<00:00, 28.83it/s]


100%|██████████| 2357/2357 [03:43<00:00, 10.56it/s]
[I 2023-04-23 05:43:13,211] Trial 24 finished with value: 0.1695119713150707 and parameters: {'emb_size': 32, 'batch_size': 256, 'lr': 0.0006366285017414498, 'l2_reg_coef': 0.09063752099202302}. Best is trial 24 with value: 0.1695119713150707.


Test SLRC (calculate scores for different seeds):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 2358/2358 [00:14<00:00, 163.95it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:15<00:00, 153.46it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:15<00:00, 152.40it/s]


In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": SLRC(
            base_model_class=BPR,
            base_model_config={
                "emb_size": study.best_params["emb_size"],
                "user_num": corpus.n_users,
                "item_num": corpus.n_items,
                "click_num": corpus.n_clicks
            },
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___0
Epoch 1:


Batch loss = 0.57551: 100%|██████████| 7530/7530 [04:19<00:00, 29.06it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [03:19<00:00, 11.79it/s]


 {'precision': 0.118879932117098, 'recall': 0.18208650031932486, 'ndcg': 0.16937464958948809}
Epoch 2:


...


Batch loss = 0.449101: 100%|██████████| 7530/7530 [04:21<00:00, 28.81it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [03:21<00:00, 11.68it/s]


 {'precision': 0.11658888417479846, 'recall': 0.1798352159912159, 'ndcg': 0.16606146931135962}



100%|██████████| 2357/2357 [03:23<00:00, 11.61it/s]


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.11917691981332204,
 'recall': 0.1727385625823469,
 'ndcg': 0.16745920832459504}